In [1]:
import random

import numpy as np
import torch
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import tqdm
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [2]:
data = np.load(r'D:\heat_wave\WEIO\expand_WEIO\other_variables\wind_93_19_pacific_warm_pool_area.npz')
print(data.files)  # ['time', 'lat', 'lon', 'eastward_wind', 'northward_wind', 'wind_vector_divergence', 'wind_stress', 'surface_downward_eastward_stress', 'surface_downward_northward_stress', 'wind_stress_curl', 'wind_stress_divergence', 'wind_speed_rms', 'eastward_wind_rms', 'northward_wind_rms', 'sampling_length', 'surface_type']

eastward_wind = data['eastward_wind'][:, :64, :48]  # (9861, 41, 201)  #
print(eastward_wind.shape)  # (9861, 41, 201)
northward_wind = data['northward_wind'][:, :64, :48]  # (9861, 41, 201)
wind_vector_divergence = data['wind_vector_divergence'][:, :64, :48]  # (9861, 41, 201)
wind_stress = data['wind_stress'][:, :64, :48]  # (9861, 41, 201)   # 可惜这个是 0
surface_downward_eastward_stress = data['surface_downward_eastward_stress'][:, :64, :48]  # (9861, 41, 201)
surface_downward_northward_stress = data['surface_downward_northward_stress'][:, :64, :48]  # (9861, 41, 201)
wind_stress_curl = data['wind_stress_curl'][:, :64, :48]  # (9861, 41, 201)
wind_stress_divergence = data['wind_stress_divergence'][:, :64, :48]  # (9861, 41, 201)
wind_speed_rms = data['wind_speed_rms'][:, :64, :48]  # (9861, 41, 201)
eastward_wind_rms = data['eastward_wind_rms'][:, :64, :48]  # (9861, 41, 201)
northward_wind_rms = data['northward_wind_rms'][:, :64, :48]  # (9861, 41, 201)
sampling_length = data['sampling_length'][:, :64, :48]  # (9861, 41, 201)
surface_type = data['surface_type'][:, :64, :48]  # (9861, 41, 201)

['time', 'lat', 'lon', 'eastward_wind', 'northward_wind', 'wind_vector_divergence', 'wind_stress', 'surface_downward_eastward_stress', 'surface_downward_northward_stress', 'wind_stress_curl', 'wind_stress_divergence', 'wind_speed_rms', 'eastward_wind_rms', 'northward_wind_rms', 'sampling_length', 'surface_type']
(9848, 64, 48)


In [3]:
# 首先 我们先画 MAM季节的贡献图   MAM： 3  4  5 月  即  03/01 - 05/31  当然，会做一个区域平均
# 同时，我们也会对每一年的JJA做一个平均  ---》1993 - 2019年

In [4]:
#  计算索引
import pandas as pd
import datetime

#创建时间范围
start_date = datetime.datetime(1993,1,1)
end_date = datetime.datetime(2019,12,31)
dates = pd.date_range(start = start_date, end = end_date, freq = 'D')

index = dates.get_loc(datetime.datetime(2019,8,31)) 
index

9738

In [5]:
# MAM   93
time_93_1 = 59
time_93_2 = 151
data_1 = time_93_1
data_2 = time_93_2

eastward_wind_MAM_93 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_93 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_93 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_93 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_93 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_93 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_93 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_93 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_93 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_93 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_93 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_93 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_93 = surface_type[data_1 : data_2,:,:]

In [6]:
# MAM   94
time_94_1 = 424
time_94_2 = 516
data_1 = time_94_1
data_2 = time_94_2

eastward_wind_MAM_94 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_94 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_94 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_94 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_94 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_94 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_94 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_94 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_94 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_94 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_94 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_94 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_94 = surface_type[data_1 : data_2,:,:]

In [7]:
# MAM   95
time_95_1 = 789
time_95_2 = 881
data_1 = time_95_1
data_2 = time_95_2

eastward_wind_MAM_95 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_95 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_95 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_95 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_95 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_95 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_95 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_95 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_95 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_95 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_95 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_95 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_95 = surface_type[data_1 : data_2,:,:]

In [8]:
# MAM   96
time_96_1 = 1155
time_96_2 = 1247
data_1 = time_96_1
data_2 = time_96_2

eastward_wind_MAM_96 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_96 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_96 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_96 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_96 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_96 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_96 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_96 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_96 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_96 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_96 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_96 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_96 = surface_type[data_1 : data_2,:,:]

In [9]:
# MAM   97
time_97_1 = 1520
time_97_2 = 1611
data_1 = time_97_1
data_2 = time_97_2

eastward_wind_MAM_97 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_97 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_97 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_97 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_97 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_97 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_97 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_97 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_97 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_97 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_97 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_97 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_97 = surface_type[data_1 : data_2,:,:]

In [10]:
# MAM   98
time_98_1 = 1885
time_98_2 = 1977
data_1 = time_98_1
data_2 = time_98_2

eastward_wind_MAM_98 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_98 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_98 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_98 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_98 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_98 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_98 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_98 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_98 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_98 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_98 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_98 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_98 = surface_type[data_1 : data_2,:,:]

In [11]:
# MAM   99
time_99_1 = 2250
time_99_2 = 2342
data_1 = time_99_1
data_2 = time_99_2

eastward_wind_MAM_99 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_99 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_99 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_99 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_99 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_99 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_99 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_99 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_99 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_99 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_99 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_99 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_99 = surface_type[data_1 : data_2,:,:]

In [13]:
# MAM   00
time_00_1 = 2616
time_00_2 = 2708
data_1 = time_00_1
data_2 = time_00_2

eastward_wind_MAM_00 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_00 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_00 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_00 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_00 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_00 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_00 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_00 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_00 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_00 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_00 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_00 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_00 = surface_type[data_1 : data_2,:,:]

In [14]:
# MAM   01
time_01_1 = 2981
time_01_2 = 3073
data_1 = time_01_1
data_2 = time_01_2

eastward_wind_MAM_01 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_01 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_01 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_01 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_01 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_01 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_01 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_01 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_01 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_01 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_01 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_01 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_01 = surface_type[data_1 : data_2,:,:]

In [15]:
# MAM   02
time_02_1 = 3346
time_02_2 = 3438
data_1 = time_02_1
data_2 = time_02_2

eastward_wind_MAM_02 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_02 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_02 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_02 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_02 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_02 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_02 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_02 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_02 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_02 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_02 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_02 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_02 = surface_type[data_1 : data_2,:,:]

In [16]:
# MAM   03
time_03_1 = 3711
time_03_2 = 3803
data_1 = time_03_1
data_2 = time_03_2

eastward_wind_MAM_03 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_03 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_03 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_03 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_03 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_03 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_03 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_03 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_03 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_03 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_03 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_03 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_03 = surface_type[data_1 : data_2,:,:]

In [17]:
# MAM   04
time_04_1 = 4077
time_04_2 = 4169
data_1 = time_04_1
data_2 = time_04_2

eastward_wind_MAM_04 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_04 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_04 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_04 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_04 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_04 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_04 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_04 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_04 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_04 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_04 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_04 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_04 = surface_type[data_1 : data_2,:,:]

In [18]:
# MAM   05
time_05_1 = 4442
time_05_2 = 4534
data_1 = time_05_1
data_2 = time_05_2

eastward_wind_MAM_05 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_05 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_05 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_05 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_05 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_05 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_05 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_05 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_05 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_05 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_05 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_05 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_05 = surface_type[data_1 : data_2,:,:]

In [19]:
# MAM   06
time_06_1 = 4807
time_06_2 = 4899
data_1 = time_06_1
data_2 = time_06_2

eastward_wind_MAM_06 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_06 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_06 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_06 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_06 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_06 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_06 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_06 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_06 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_06 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_06 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_06 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_06 = surface_type[data_1 : data_2,:,:]

In [20]:
# MAM   07
time_07_1 = 5172
time_07_2 = 5264
data_1 = time_07_1
data_2 = time_07_2

eastward_wind_MAM_07 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_07 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_07 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_07 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_07 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_07 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_07 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_07 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_07 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_07 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_07 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_07 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_07 = surface_type[data_1 : data_2,:,:]

In [21]:
# MAM   08
time_08_1 = 5538
time_08_2 = 5630
data_1 = time_08_1
data_2 = time_08_2

eastward_wind_MAM_08 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_08 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_08 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_08 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_08 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_08 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_08 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_08 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_08 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_08 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_08 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_08 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_08 = surface_type[data_1 : data_2,:,:]

In [22]:
# MAM   09
time_09_1 = 5903
time_09_2 = 5995
data_1 = time_09_1
data_2 = time_09_2

eastward_wind_MAM_09 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_09 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_09 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_09 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_09 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_09 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_09 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_09 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_09 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_09 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_09 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_09 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_09 = surface_type[data_1 : data_2,:,:]

In [23]:
# MAM   10
time_10_1 = 6268
time_10_2 = 6360
data_1 = time_10_1
data_2 = time_10_2

eastward_wind_MAM_10 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_10 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_10 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_10 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_10 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_10 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_10 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_10 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_10 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_10 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_10 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_10 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_10 = surface_type[data_1 : data_2,:,:]

In [24]:
# MAM   11
time_11_1 = 6633
time_11_2 = 6725
data_1 = time_11_1
data_2 = time_11_2

eastward_wind_MAM_11 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_11 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_11 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_11 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_11 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_11 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_11 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_11 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_11 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_11 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_11 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_11 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_11 = surface_type[data_1 : data_2,:,:]

In [25]:
# MAM   12
time_12_1 = 6999
time_12_2 = 7091
data_1 = time_12_1
data_2 = time_12_2

eastward_wind_MAM_12 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_12 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_12 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_12 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_12 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_12 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_12 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_12 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_12 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_12 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_12 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_12 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_12 = surface_type[data_1 : data_2,:,:]

In [26]:
# MAM   13
time_13_1 = 7364
time_13_2 = 7455
data_1 = time_13_1
data_2 = time_13_2

eastward_wind_MAM_13 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_13 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_13 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_13 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_13 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_13 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_13 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_13 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_13 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_13 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_13 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_13 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_13 = surface_type[data_1 : data_2,:,:]

In [27]:
# MAM   14
time_14_1 = 7729
time_14_2 = 7821
data_1 = time_14_1
data_2 = time_14_2

eastward_wind_MAM_14 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_14 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_14 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_14 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_14 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_14 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_14 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_14 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_14 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_14 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_14 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_14 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_14 = surface_type[data_1 : data_2,:,:]

In [28]:
# MAM   15
time_15_1 = 8094
time_15_2 = 8185
data_1 = time_15_1
data_2 = time_15_2

eastward_wind_MAM_15 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_15 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_15 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_15 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_15 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_15 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_15 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_15 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_15 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_15 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_15 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_15 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_15 = surface_type[data_1 : data_2,:,:]

In [29]:
# MAM   16
time_16_1 = 8460
time_16_2 = 8552
data_1 = time_16_1
data_2 = time_16_2

eastward_wind_MAM_16 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_16 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_16 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_16 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_16 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_16 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_16 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_16 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_16 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_16 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_16 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_16 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_16 = surface_type[data_1 : data_2,:,:]

In [30]:
# MAM   17
time_17_1 = 8825
time_17_2 = 8917
data_1 = time_17_1
data_2 = time_17_2

eastward_wind_MAM_17 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_17 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_17 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_17 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_17 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_17 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_17 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_17 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_17 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_17 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_17 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_17 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_17 = surface_type[data_1 : data_2,:,:]

In [31]:
# MAM   18
time_18_1 = 9190
time_18_2 = 9280
data_1 = time_18_1
data_2 = time_18_2

eastward_wind_MAM_18 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_18 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_18 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_18 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_18 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_18 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_18 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_18 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_18 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_18 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_18 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_18 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_18 = surface_type[data_1 : data_2,:,:]

In [32]:
# MAM   19
time_19_1 = 9542
time_19_2 = 9633
data_1 = time_19_1
data_2 = time_19_2

eastward_wind_MAM_19 = eastward_wind[data_1 : data_2,:,:]
northward_wind_MAM_19 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_MAM_19 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_MAM_19 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_MAM_19 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_MAM_19 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_MAM_19 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_MAM_19 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_MAM_19 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_MAM_19 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_MAM_19 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_MAM_19 = sampling_length[data_1 : data_2,:,:]
surface_type_MAM_19 = surface_type[data_1 : data_2,:,:]

In [33]:
wind_stress_MAM = np.concatenate((wind_stress_MAM_93, wind_stress_MAM_94, wind_stress_MAM_95,
                                 wind_stress_MAM_96, wind_stress_MAM_97, wind_stress_MAM_98, wind_stress_MAM_99,
                                 wind_stress_MAM_00, wind_stress_MAM_01, wind_stress_MAM_02, wind_stress_MAM_03,
                                 wind_stress_MAM_04, wind_stress_MAM_05, wind_stress_MAM_06, wind_stress_MAM_07,
                                 wind_stress_MAM_08, wind_stress_MAM_09, wind_stress_MAM_10, wind_stress_MAM_11,
                                 wind_stress_MAM_12, wind_stress_MAM_13, wind_stress_MAM_14, wind_stress_MAM_15,
                                 wind_stress_MAM_16, wind_stress_MAM_17, wind_stress_MAM_18, wind_stress_MAM_19), axis = 0)

In [34]:
surface_type_MAM = np.concatenate((surface_type_MAM_93, surface_type_MAM_94, surface_type_MAM_95,
                                  surface_type_MAM_96, surface_type_MAM_97, surface_type_MAM_98, surface_type_MAM_99,
                                  surface_type_MAM_00, surface_type_MAM_01, surface_type_MAM_02, surface_type_MAM_03,
                                  surface_type_MAM_04, surface_type_MAM_05, surface_type_MAM_06, surface_type_MAM_07,
                                  surface_type_MAM_08, surface_type_MAM_09, surface_type_MAM_10, surface_type_MAM_11,
                                  surface_type_MAM_12, surface_type_MAM_13, surface_type_MAM_14, surface_type_MAM_15,
                                  surface_type_MAM_16, surface_type_MAM_17, surface_type_MAM_18, surface_type_MAM_19), axis = 0)

In [35]:
eastward_wind_MAM = np.concatenate((eastward_wind_MAM_93, eastward_wind_MAM_94, eastward_wind_MAM_95,
                                   eastward_wind_MAM_96, eastward_wind_MAM_97, eastward_wind_MAM_98,
                                   eastward_wind_MAM_99, eastward_wind_MAM_00, eastward_wind_MAM_01,
                                   eastward_wind_MAM_02, eastward_wind_MAM_03, eastward_wind_MAM_04,
                                   eastward_wind_MAM_05, eastward_wind_MAM_06, eastward_wind_MAM_07,
                                   eastward_wind_MAM_08, eastward_wind_MAM_09, eastward_wind_MAM_10,
                                   eastward_wind_MAM_11, eastward_wind_MAM_12, eastward_wind_MAM_13,
                                   eastward_wind_MAM_14, eastward_wind_MAM_15, eastward_wind_MAM_16,
                                   eastward_wind_MAM_17, eastward_wind_MAM_18, eastward_wind_MAM_19), axis = 0)

In [36]:
wind_stress_curl_MAM = np.concatenate((wind_stress_curl_MAM_93, wind_stress_curl_MAM_94, wind_stress_curl_MAM_95,
                                      wind_stress_curl_MAM_96, wind_stress_curl_MAM_97, wind_stress_curl_MAM_98, 
                                      wind_stress_curl_MAM_99, wind_stress_curl_MAM_00, wind_stress_curl_MAM_01,
                                      wind_stress_curl_MAM_02, wind_stress_curl_MAM_03, wind_stress_curl_MAM_04, 
                                      wind_stress_curl_MAM_05, wind_stress_curl_MAM_06, wind_stress_curl_MAM_07,
                                      wind_stress_curl_MAM_08, wind_stress_curl_MAM_09, wind_stress_curl_MAM_10,
                                      wind_stress_curl_MAM_11, wind_stress_curl_MAM_12, wind_stress_curl_MAM_13,
                                      wind_stress_curl_MAM_14, wind_stress_curl_MAM_15, wind_stress_curl_MAM_16,
                                      wind_stress_curl_MAM_17, wind_stress_curl_MAM_18, wind_stress_curl_MAM_19), axis = 0)

In [37]:
northward_wind_MAM = np.concatenate((northward_wind_MAM_93, northward_wind_MAM_94, northward_wind_MAM_95,
                                    northward_wind_MAM_96, northward_wind_MAM_97, northward_wind_MAM_98,
                                    northward_wind_MAM_99, northward_wind_MAM_00, northward_wind_MAM_01,
                                    northward_wind_MAM_02, northward_wind_MAM_03, northward_wind_MAM_04, 
                                    northward_wind_MAM_05, northward_wind_MAM_06, northward_wind_MAM_07,
                                    northward_wind_MAM_08, northward_wind_MAM_09, northward_wind_MAM_10,
                                    northward_wind_MAM_11, northward_wind_MAM_12, northward_wind_MAM_13,
                                    northward_wind_MAM_14, northward_wind_MAM_15, northward_wind_MAM_16,
                                    northward_wind_MAM_17, northward_wind_MAM_18, northward_wind_MAM_19), axis = 0)

In [38]:
wind_speed_rms_MAM = np.concatenate((wind_speed_rms_MAM_93, wind_speed_rms_MAM_94, wind_speed_rms_MAM_95,
                                    wind_speed_rms_MAM_96, wind_speed_rms_MAM_97, wind_speed_rms_MAM_98,
                                    wind_speed_rms_MAM_99, wind_speed_rms_MAM_00, wind_speed_rms_MAM_01,
                                    wind_speed_rms_MAM_02, wind_speed_rms_MAM_03, wind_speed_rms_MAM_04,
                                    wind_speed_rms_MAM_05, wind_speed_rms_MAM_06, wind_speed_rms_MAM_07,
                                    wind_speed_rms_MAM_08, wind_speed_rms_MAM_09, wind_speed_rms_MAM_10,
                                    wind_speed_rms_MAM_11, wind_speed_rms_MAM_12, wind_speed_rms_MAM_13,
                                    wind_speed_rms_MAM_14, wind_speed_rms_MAM_15, wind_speed_rms_MAM_16,
                                    wind_speed_rms_MAM_17, wind_speed_rms_MAM_18, wind_speed_rms_MAM_19), axis = 0)

In [39]:
sampling_length_MAM = np.concatenate((sampling_length_MAM_93, sampling_length_MAM_94, sampling_length_MAM_95,
                                     sampling_length_MAM_96, sampling_length_MAM_97, sampling_length_MAM_98,
                                     sampling_length_MAM_99, sampling_length_MAM_00, sampling_length_MAM_01,
                                     sampling_length_MAM_02, sampling_length_MAM_03, sampling_length_MAM_04,
                                     sampling_length_MAM_05, sampling_length_MAM_06, sampling_length_MAM_07,
                                     sampling_length_MAM_08, sampling_length_MAM_09, sampling_length_MAM_10,
                                     sampling_length_MAM_11, sampling_length_MAM_12, sampling_length_MAM_13,
                                     sampling_length_MAM_14, sampling_length_MAM_15, sampling_length_MAM_16,
                                     sampling_length_MAM_17, sampling_length_MAM_18, sampling_length_MAM_19), axis = 0)

In [40]:
eastward_wind_rms_MAM = np.concatenate((eastward_wind_rms_MAM_93, eastward_wind_rms_MAM_94, eastward_wind_rms_MAM_95,
                                       eastward_wind_rms_MAM_96, eastward_wind_rms_MAM_97, eastward_wind_rms_MAM_98,
                                       eastward_wind_rms_MAM_99, eastward_wind_rms_MAM_00, eastward_wind_rms_MAM_01,
                                       eastward_wind_rms_MAM_02, eastward_wind_rms_MAM_03, eastward_wind_rms_MAM_04,
                                       eastward_wind_rms_MAM_05, eastward_wind_rms_MAM_06, eastward_wind_rms_MAM_07,
                                       eastward_wind_rms_MAM_08, eastward_wind_rms_MAM_09, eastward_wind_rms_MAM_10,
                                       eastward_wind_rms_MAM_11, eastward_wind_rms_MAM_12, eastward_wind_rms_MAM_13,
                                       eastward_wind_rms_MAM_14, eastward_wind_rms_MAM_15, eastward_wind_rms_MAM_16,
                                       eastward_wind_rms_MAM_17, eastward_wind_rms_MAM_18, eastward_wind_rms_MAM_19), axis = 0)

In [41]:
northward_wind_rms_MAM = np.concatenate((northward_wind_rms_MAM_93, northward_wind_rms_MAM_94,
                                        northward_wind_rms_MAM_95, northward_wind_rms_MAM_96,northward_wind_rms_MAM_97,
                                        northward_wind_rms_MAM_98, northward_wind_rms_MAM_99, northward_wind_rms_MAM_00,
                                        northward_wind_rms_MAM_01, northward_wind_rms_MAM_02, northward_wind_rms_MAM_03,
                                        northward_wind_rms_MAM_04, northward_wind_rms_MAM_05, northward_wind_rms_MAM_06,
                                        northward_wind_rms_MAM_07, northward_wind_rms_MAM_08, northward_wind_rms_MAM_09,
                                        northward_wind_rms_MAM_10, northward_wind_rms_MAM_11, northward_wind_rms_MAM_12,
                                        northward_wind_rms_MAM_13, northward_wind_rms_MAM_14, northward_wind_rms_MAM_15,
                                        northward_wind_rms_MAM_16, northward_wind_rms_MAM_17, northward_wind_rms_MAM_18,
                                        northward_wind_rms_MAM_19), axis = 0)

In [42]:
wind_stress_divergence_MAM = np.concatenate((wind_stress_divergence_MAM_93, wind_stress_divergence_MAM_94,
                                            wind_stress_divergence_MAM_95, wind_stress_divergence_MAM_96, 
                                            wind_stress_divergence_MAM_97, wind_stress_divergence_MAM_98,
                                            wind_stress_divergence_MAM_99, wind_stress_divergence_MAM_00,
                                            wind_stress_divergence_MAM_01, wind_stress_divergence_MAM_02,
                                            wind_stress_divergence_MAM_03, wind_stress_divergence_MAM_04,
                                            wind_stress_divergence_MAM_05, wind_stress_divergence_MAM_06, 
                                            wind_stress_divergence_MAM_07, wind_stress_divergence_MAM_08,
                                            wind_stress_divergence_MAM_09, wind_stress_divergence_MAM_10,
                                            wind_stress_divergence_MAM_11, wind_stress_divergence_MAM_12,
                                            wind_stress_divergence_MAM_13, wind_stress_divergence_MAM_14,
                                            wind_stress_divergence_MAM_15, wind_stress_divergence_MAM_16,
                                            wind_stress_divergence_MAM_17, wind_stress_divergence_MAM_18, wind_stress_divergence_MAM_19), axis = 0)

In [43]:
wind_vector_divergence_MAM = np.concatenate((wind_vector_divergence_MAM_93, wind_vector_divergence_MAM_94,
                                            wind_vector_divergence_MAM_95, wind_vector_divergence_MAM_96,
                                            wind_vector_divergence_MAM_97, wind_vector_divergence_MAM_98,
                                            wind_vector_divergence_MAM_99, wind_vector_divergence_MAM_00,
                                            wind_vector_divergence_MAM_01, wind_vector_divergence_MAM_02,
                                            wind_vector_divergence_MAM_03, wind_vector_divergence_MAM_04,
                                            wind_vector_divergence_MAM_05, wind_vector_divergence_MAM_06,
                                            wind_vector_divergence_MAM_07, wind_vector_divergence_MAM_08,
                                            wind_vector_divergence_MAM_09, wind_vector_divergence_MAM_10,
                                            wind_vector_divergence_MAM_11, wind_vector_divergence_MAM_12,
                                            wind_vector_divergence_MAM_13, wind_vector_divergence_MAM_14,
                                            wind_vector_divergence_MAM_15, wind_vector_divergence_MAM_16,
                                            wind_vector_divergence_MAM_17, wind_vector_divergence_MAM_18, wind_vector_divergence_MAM_19), axis = 0)

In [44]:
surface_downward_eastward_stress_MAM = np.concatenate((surface_downward_eastward_stress_MAM_93,
                                                      surface_downward_eastward_stress_MAM_94, surface_downward_eastward_stress_MAM_95,
                                                      surface_downward_eastward_stress_MAM_96, surface_downward_eastward_stress_MAM_97,
                                                      surface_downward_eastward_stress_MAM_98, surface_downward_eastward_stress_MAM_99,
                                                      surface_downward_eastward_stress_MAM_00, surface_downward_eastward_stress_MAM_01,
                                                      surface_downward_eastward_stress_MAM_02, surface_downward_eastward_stress_MAM_03,
                                                      surface_downward_eastward_stress_MAM_04, surface_downward_eastward_stress_MAM_05,
                                                      surface_downward_eastward_stress_MAM_06, surface_downward_eastward_stress_MAM_07,
                                                      surface_downward_eastward_stress_MAM_08, surface_downward_eastward_stress_MAM_09,
                                                      surface_downward_eastward_stress_MAM_10, surface_downward_eastward_stress_MAM_11,
                                                      surface_downward_eastward_stress_MAM_12, surface_downward_eastward_stress_MAM_13,
                                                      surface_downward_eastward_stress_MAM_14, surface_downward_eastward_stress_MAM_15,
                                                      surface_downward_eastward_stress_MAM_16, surface_downward_eastward_stress_MAM_17,
                                                      surface_downward_eastward_stress_MAM_18, surface_downward_eastward_stress_MAM_19), axis = 0)

In [45]:
surface_downward_northward_stress_MAM = np.concatenate((surface_downward_northward_stress_MAM_93,
                                                       surface_downward_northward_stress_MAM_94, surface_downward_northward_stress_MAM_95,
                                                       surface_downward_northward_stress_MAM_96, surface_downward_northward_stress_MAM_97,
                                                       surface_downward_northward_stress_MAM_98, surface_downward_northward_stress_MAM_99,
                                                       surface_downward_northward_stress_MAM_00, surface_downward_northward_stress_MAM_01,
                                                       surface_downward_northward_stress_MAM_02, surface_downward_northward_stress_MAM_03,
                                                       surface_downward_northward_stress_MAM_04, surface_downward_northward_stress_MAM_05,
                                                       surface_downward_northward_stress_MAM_06, surface_downward_northward_stress_MAM_07,
                                                       surface_downward_northward_stress_MAM_08, surface_downward_northward_stress_MAM_09,
                                                       surface_downward_northward_stress_MAM_10, surface_downward_northward_stress_MAM_11,
                                                       surface_downward_northward_stress_MAM_12, surface_downward_northward_stress_MAM_13,
                                                       surface_downward_northward_stress_MAM_14, surface_downward_northward_stress_MAM_15,
                                                       surface_downward_northward_stress_MAM_16, surface_downward_northward_stress_MAM_17,
                                                       surface_downward_northward_stress_MAM_18, surface_downward_northward_stress_MAM_19), axis = 0)

In [48]:
np.savez(r'D:\heat_wave\WEIO\expand_WEIO\other_variables\last\wind_93_19_WEIO_area_MAM.npz',
        wind_stress_MAM = wind_stress_MAM, surface_type_MAM = surface_type_MAM, eastward_wind_MAM = eastward_wind_MAM,
        wind_stress_curl_MAM = wind_stress_curl_MAM, northward_wind_MAM = northward_wind_MAM, 
        wind_speed_rms_MAM = wind_speed_rms_MAM, sampling_length_MAM = sampling_length_MAM,
        eastward_wind_rms_MAM = eastward_wind_rms_MAM, northward_wind_rms_MAM = northward_wind_rms_MAM,
        surface_downward_eastward_stress_MAM = surface_downward_eastward_stress_MAM,
        surface_downward_northward_stress_MAM = surface_downward_northward_stress_MAM)